# steps

1. download freesurfer (https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall)
2. download pysurfer (https://pysurfer.github.io/install.html)

  - mayavi installation might take a bit
  
  
3. run the terminal export source commands (or add them to your ~.profile or bashrc

  - ```export FREESURFER_HOME=/Applications/freesurfer```
  - ```source $FREESURFER_HOME/SetUpFreeSurfer.sh```
  

notes: 
  - if you have your data on the server, make sure to mount it! 
  - this is just a working example and could probably be optimized a bit more.



## random data example

In [1]:
%gui qt
from mayavi import mlab
import mayavi.mlab
import pyface.qt
import os
from os.path import expanduser
from surfer import Brain
import numpy as np
import nibabel as nib

In [2]:
randActivity = np.random.rand((40962))
randActivity.shape

(40962,)

In [3]:
"""

Define the three important variables.
Note that these are the first three positional arguments
in tksurfer (and pysurfer for that matter).
"""

subject_id = 'fsaverage'
hemi = 'lh' #'both' 
surf = 'inflated' #'smoothwm'# 'inflated' 

"""

Set up the environment defs
"""

#mounted subject directory form fmriprep output
os.environ['SUBJECTS_DIR'] = expanduser('/Volumes/rmasis/ProjectName/data/BIDS/derivatives/freesurfer')

#can also be:
#os.environ['SUBJECTS_DIR'] = expanduser('/Applications/freesurfer//subjects')
#since freesurfer directories are super standardized, you can use this one too.
#but, it might be safe to use the fmriprep output directory since it will also include subject specific info (fmriprep specific)


"""

Call the Brain object constructor with these
parameters to initialize the visualization session.
"""

brain = Brain(subject_id, hemi, surf, cortex='high_contrast',alpha=.2)

#works!
#the name argument is whatever you want it to be. 
brain.add_overlay(randActivity ,min =0, max=np.max(randActivity), name= 'randActivity_lh', hemi='lh')

mayavi.mlab.show()


## Example with statistical data

In [4]:
import deepdish as dd
import numpy as np

analypath = '/Volumes/norman/rmasis/MemPal/analysis/PythonData/'

data = dd.io.load(analypath + 'isc.h5' )
data['left'] = np.nan_to_num(data['left'])

np.max(data['left'])

#data['left'].shape

0.61835719447068105

In [5]:
databinary = np.where(data['left']>.2,1,0)

print(databinary.shape) #standard fsaverage6 hemisphere size

(40962,)


In [6]:
subject_id = 'fsaverage'
hemi = 'lh'
surf = 'inflated'

os.environ['SUBJECTS_DIR'] = expanduser('/Volumes/rmasis/ProjectName/data/BIDS/derivatives/freesurfer')

brain = Brain(subject_id, hemi, surf)

#works!
#brain.add_overlay(data['left'],min = 0.0, max=np.max(data['left']), name= 'ang_corr_lh', hemi='lh')

#works!
brain.add_overlay(databinary,min = 0.0, max=1, name= 'ang_corr_lh', hemi='lh')

mayavi.mlab.show()

## Labeling pysurfer example

In [10]:
subject_id = "fsaverage"
hemi = "lh"
surf = "inflated"

"""
Bring up the visualization.
"""
os.environ['SUBJECTS_DIR'] = expanduser('/Volumes/rmasis/ProjectName/data/BIDS/derivatives/freesurfer')

brain = Brain(subject_id, hemi, surf, background="white")

"""
Read in the automatic parcellation of sulci and gyri.
"""
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".aparc.a2009s.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

"""
Make a random vector of scalar data corresponding to a value for each region in
the parcellation.

"""
rs = np.random.RandomState(4)
print('rs: ', np.shape(rs))
roi_data = rs.uniform(.5, .8, size=len(names))
print('roi data: ', np.shape(roi_data))


"""
Make a vector containing the data point at each vertex.
"""
vtx_data = roi_data[labels]
print('vtx data: ', np.shape(vtx_data))


"""
Handle vertices that are not defined in the annotation.
"""
vtx_data[labels == -1] = -1

"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
brain.add_data(vtx_data, .5, .75, thresh=0, colormap="rocket", alpha=.8)

mayavi.mlab.show()

('rs: ', ())
('roi data: ', (76,))
('vtx data: ', (163842,))
